## Tanzania's Waterwell Problem
This project aims to provide an overview of the water well situation in Tanzania to the Tanzanian government. The focus is on mitigating a water crisis by predicting water wells that will soon be in need of repair and planning future water well locations for maximum effectiveness.


In [1]:
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import seaborn as sns
import pandas as pd
import numpy as np
import sys

import sklearn as sk
from sklearn import metrics
from sklearn.model_selection import train_test_split, ShuffleSplit, learning_curve, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [2]:
import pandas as pd

# Load the CSV files into pandas DataFrames
testvalues = pd.read_csv('/Users/jguzzo522/Desktop/WATERWELL/testsetvalues.csv')
trainingsetlabels = pd.read_csv('/Users/jguzzo522/Desktop/WATERWELL/trainingsetlabels.csv')
trainingsetvalues = pd.read_csv('/Users/jguzzo522/Desktop/WATERWELL/trainingsetvalues.csv')

# Display the loaded datasets
print("Test Set Values:")
print(testvalues.head())
print("\nTraining Set Labels:")
print(trainingsetlabels.head())
print("\nTraining Set Values:")
print(trainingsetvalues.head())


Test Set Values:
      id  amount_tsh date_recorded                  funder  gps_height  \
0  50785         0.0    2013-02-04                    Dmdd        1996   
1  51630         0.0    2013-02-04  Government Of Tanzania        1569   
2  17168         0.0    2013-02-01                     NaN        1567   
3  45559         0.0    2013-01-22              Finn Water         267   
4  49871       500.0    2013-03-27                  Bruder        1260   

    installer  longitude   latitude                 wpt_name  num_private  \
0        DMDD  35.290799  -4.059696  Dinamu Secondary School            0   
1         DWE  36.656709  -3.309214                  Kimnyak            0   
2         NaN  34.767863  -5.004344           Puma Secondary            0   
3  FINN WATER  38.058046  -9.418672           Kwa Mzee Pange            0   
4      BRUDER  35.006123 -10.950412          Kwa Mzee Turuka            0   

   ... payment_type water_quality quality_group      quantity  quantity_gro

In [3]:
# Check the number of unique IDs in each dataset
test_ids = set(testvalues['id'])
training_labels_ids = set(trainingsetlabels['id'])
training_values_ids = set(trainingsetvalues['id'])

print("Number of unique IDs in Test Set Values:", len(test_ids))
print("Number of unique IDs in Training Set Labels:", len(training_labels_ids))
print("Number of unique IDs in Training Set Values:", len(training_values_ids))


Number of unique IDs in Test Set Values: 14850
Number of unique IDs in Training Set Labels: 59400
Number of unique IDs in Training Set Values: 59400


In [4]:
import pandas as pd

# Load the CSV files into pandas DataFrames
training_set_labels = pd.read_csv('/Users/jguzzo522/Desktop/WATERWELL/trainingsetlabels.csv')
training_set_values = pd.read_csv('/Users/jguzzo522/Desktop/WATERWELL/trainingsetvalues.csv')

# Merge the datasets on the 'id' column
merged_data = pd.merge(training_set_values, training_set_labels, on='id', how='inner')

# Display the merged dataset
print(merged_data)


          id  amount_tsh date_recorded           funder  gps_height  \
0      69572      6000.0    2011-03-14            Roman        1390   
1       8776         0.0    2013-03-06          Grumeti        1399   
2      34310        25.0    2013-02-25     Lottery Club         686   
3      67743         0.0    2013-01-28           Unicef         263   
4      19728         0.0    2011-07-13      Action In A           0   
...      ...         ...           ...              ...         ...   
59395  60739        10.0    2013-05-03  Germany Republi        1210   
59396  27263      4700.0    2011-05-07      Cefa-njombe        1212   
59397  37057         0.0    2011-04-11              NaN           0   
59398  31282         0.0    2011-03-08            Malec           0   
59399  26348         0.0    2011-03-23       World Bank         191   

          installer  longitude   latitude              wpt_name  num_private  \
0             Roman  34.938093  -9.856322                  none    

In [5]:

# Display the first few rows of the merged dataset
merged_data.head()



,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [6]:
# Load the test set values dataset
test_set_values = pd.read_csv('/Users/jguzzo522/Desktop/WATERWELL/testsetvalues.csv')

# Print out the test set values dataset
print(test_set_values)


          id  amount_tsh date_recorded                  funder  gps_height  \
0      50785         0.0    2013-02-04                    Dmdd        1996   
1      51630         0.0    2013-02-04  Government Of Tanzania        1569   
2      17168         0.0    2013-02-01                     NaN        1567   
3      45559         0.0    2013-01-22              Finn Water         267   
4      49871       500.0    2013-03-27                  Bruder        1260   
...      ...         ...           ...                     ...         ...   
14845  39307         0.0    2011-02-24                  Danida          34   
14846  18990      1000.0    2011-03-21                    Hiap           0   
14847  28749         0.0    2013-03-04                     NaN        1476   
14848  33492         0.0    2013-02-18                 Germany         998   
14849  68707         0.0    2013-02-13  Government Of Tanzania         481   

        installer  longitude   latitude                 wpt_nam

In [7]:
# Display the first few rows of the test set values dataset
test_set_values.head()


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


## Exploring the data


In [8]:
# Get the number of unique values for each column in the merged dataset
unique_values_counts = merged_data.nunique()

# Display the number of unique values for each column
print(unique_values_counts)


id                       59400
amount_tsh                  98
date_recorded              356
funder                    1897
gps_height                2428
installer                 2145
longitude                57516
latitude                 57517
wpt_name                 37400
num_private                 65
basin                        9
subvillage               19287
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
recorded_by                  1
scheme_management           12
scheme_name               2696
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_

## Removal of "columns

The column "funder" contains a large number of unique values (1897). Attempting to analyze how 1897 different companies or individuals funded the water wells would be very time-consuming and might not provide significant insights due to the sheer volume of categories. Therefore, we have opted to remove the "funder_x" variable from our dataset to streamline our analysis and focus on other variables with potentially more significant impact on our predictive modeling.

In the initial exploration of our dataset, we identified the "subvillage" feature, which represents the geographic location of each waterpoint at a subvillage level. However, upon closer examination, we observed that this feature contains a large number of unique values (19,287). While capturing detailed geographic information is important for understanding the distribution of waterpoints, including such granular data in our predictive modeling may not yield meaningful insights and could introduce unnecessary complexity to our analysis.

The "wpt_name" column contains a large number of unique values (37,400), representing different names of waterpoints. Similar to the decision to drop the "subvillage" column, the high cardinality of this feature could potentially lead to overfitting, increased model complexity, and reduced interpretability without significantly enhancing predictive performance. Given the sheer number of distinct names and the potential for noise or redundancy in the data, it's prudent to exclude this column from our analysis to maintain simplicity and focus on features with more meaningful predictive value.

Upon examining the dataset, it is evident that both 'scheme_management' and 'scheme_name' represent the entity responsible for operating the waterpoint. However, 'scheme_management' contains only 12 unique values, whereas 'scheme_name' has 2696 unique values. Given that 'scheme_management' provides a more concise categorization with fewer unique values, we have decided to retain this variable and drop 'scheme_name' for simplicity and ease of analysis.

In [9]:
# Count the number of unique values
num_unique_values = merged_data['funder'].nunique()

# Display the number of unique values
print("Number of unique values:", num_unique_values)

# Display the unique values
unique_values = merged_data['funder'].unique()
print("Unique values:")
print(unique_values)


Number of unique values: 1897
Unique values:
['Roman' 'Grumeti' 'Lottery Club' ... 'Dina' 'Brown' 'Samlo']


In [10]:
# Drop the "funder" column
merged_data.drop(columns=['funder'], inplace=True)


In the initial exploration of our dataset, we identified the "subvillage" feature, which represents the geographic location of each waterpoint at a subvillage level. However, upon closer examination, we observed that this feature contains a large number of unique values (19,287). While capturing detailed geographic information is important for understanding the distribution of waterpoints, including such granular data in our predictive modeling may not yield meaningful insights and could introduce unnecessary complexity to our analysis.


In [11]:
# Drop the "subvillage" column
merged_data.drop(columns=['subvillage'], inplace=True)


The "wpt_name" column contains a large number of unique values (37,400), representing different names of waterpoints. Similar to the decision to drop the "subvillage" column, the high cardinality of this feature could potentially lead to overfitting, increased model complexity, and reduced interpretability without significantly enhancing predictive performance. Given the sheer number of distinct names and the potential for noise or redundancy in the data, it's prudent to exclude this column from our analysis to maintain simplicity and focus on features with more meaningful predictive value.

In [12]:
# Drop the "wpt_name" column
merged_data.drop(columns=['wpt_name'], inplace=True)


Upon examining the dataset, it is evident that both 'scheme_management' and 'scheme_name' represent the entity responsible for operating the waterpoint. However, 'scheme_management' contains only 12 unique values, whereas 'scheme_name' has 2696 unique values. Given that 'scheme_management' provides a more concise categorization with fewer unique values, we have decided to retain this variable and drop 'scheme_name' for simplicity and ease of analysis.

In [13]:
# Drop the "subvillage" column
merged_data.drop(columns=['scheme_name'], inplace=True)

In [14]:
# Check the number of unique values in the 'installer' column
num_unique_installers = len(merged_data['installer'].unique())
print("Number of unique installers:", num_unique_installers)

# Display some of the unique values in the 'installer' column
unique_installers = merged_data['installer'].unique()
print("Unique installers:", unique_installers[:10])  # Displaying the first 10 unique values


Number of unique installers: 2146
Unique installers: ['Roman' 'GRUMETI' 'World vision' 'UNICEF' 'Artisan' 'DWE' 'DWSP'
 'Water Aid' 'Private' 'DANIDA']


The 'installer' column contains a large number of unique values (2146), indicating a wide variety of organizations or individuals responsible for installing waterpoints. Analyzing each unique installer would be cumbersome and may not significantly contribute to our predictive modeling efforts. Furthermore, the identity of the installer might not have a direct impact on the functionality of waterpoints. Therefore, removing this column streamlines our dataset, focusing on more relevant features for predictive modeling.

In [15]:
# Drop the "installer" column
merged_data.drop(columns=['installer'], inplace=True)


In [16]:
# Get the number of unique values for each column in the merged dataset
unique_values_counts = merged_data.nunique()

# Display the number of unique values for each column
print(unique_values_counts)

id                       59400
amount_tsh                  98
date_recorded              356
gps_height                2428
longitude                57516
latitude                 57517
num_private                 65
basin                        9
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
recorded_by                  1
scheme_management           12
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity                     5
quantity_group               5
source                      10
source_type                  7
source_c

In [17]:
unique_management_values = merged_data['management'].unique()
unique_management_group_values = merged_data['management_group'].unique()

print("Unique values in 'management':")
print(unique_management_values)

print("\nUnique values in 'management_group':")
print(unique_management_group_values)


Unique values in 'management':
['vwc' 'wug' 'other' 'private operator' 'water board' 'wua' 'company'
 'water authority' 'parastatal' 'unknown' 'other - school' 'trust']

Unique values in 'management_group':
['user-group' 'other' 'commercial' 'parastatal' 'unknown']


Upon examining the dataset, it became apparent that both 'management' and 'management_group' convey information about the entity responsible for operating the waterpoint. However, 'management' encompasses a wider range of specific entities, resulting in 12 unique values, while 'management_group' provides a broader categorization with 5 unique values. Given that 'management_group' offers a more generalized classification and is easier to interpret, especially for predictive modeling, we have chosen to retain it and discard 'management'. This decision streamlines the dataset, making it more manageable for analysis and modeling purposes.

In [18]:
# Drop the "managment" column
merged_data.drop(columns=['management'], inplace=True)

In [19]:
# Unique values in 'waterpoint_type'
print("Unique values in 'waterpoint_type':")
print(merged_data['waterpoint_type'].unique())

# Unique values in 'waterpoint_type_group'
print("\nUnique values in 'waterpoint_type_group':")
print(merged_data['waterpoint_type_group'].unique())


Unique values in 'waterpoint_type':
['communal standpipe' 'communal standpipe multiple' 'hand pump' 'other'
 'improved spring' 'cattle trough' 'dam']

Unique values in 'waterpoint_type_group':
['communal standpipe' 'hand pump' 'other' 'improved spring'
 'cattle trough' 'dam']


Upon examining the unique values in the 'waterpoint_type' and 'waterpoint_type_group' columns, it's apparent that both columns represent categories of waterpoints, but 'waterpoint_type' provides more detailed distinctions with seven unique values, while 'waterpoint_type_group' aggregates these into six categories.

Considering the potential significance of the distinctions between different types of waterpoints, it may be beneficial to retain the column with more granular information ('waterpoint_type') and drop the column with fewer unique values ('waterpoint_type_group'). By doing so, we maintain a more detailed representation of the types of waterpoints in the dataset, which could potentially offer more insights during analysis or modeling.

Therefore, we have decided to drop the 'waterpoint_type_group' column to streamline the dataset while preserving the detailed categorization provided by the 'waterpoint_type' column.






In [20]:
# Drop the "waterpoint_type_group" column
merged_data.drop(columns=['waterpoint_type_group'], inplace=True)

In [21]:
print("Unique values in 'quantity':")
print(merged_data['quantity'].unique())

print("\nUnique values in 'quantity_group':")
print(merged_data['quantity_group'].unique())
print("Number of NaN values in 'quantity':", merged_data['quantity'].isna().sum())
print("Number of NaN values in 'quantity_group':", merged_data['quantity_group'].isna().sum())



Unique values in 'quantity':
['enough' 'insufficient' 'dry' 'seasonal' 'unknown']

Unique values in 'quantity_group':
['enough' 'insufficient' 'dry' 'seasonal' 'unknown']
Number of NaN values in 'quantity': 0
Number of NaN values in 'quantity_group': 0


Upon examination, it was clear these two columns were identical. Therefore it was appropriate to remove one column and so 'quantity_group' was dropped.

In [22]:
# Drop the "quantity_group" column
merged_data.drop(columns=['quantity_group'], inplace=True)

In [23]:
print("Unique values in 'water_quality':")
print(merged_data['water_quality'].unique())

print("\nUnique values in 'quality_group':")
print(merged_data['quality_group'].unique())
print("Number of NaN values in 'water_quality':", merged_data['water_quality'].isna().sum())
print("Number of NaN values in 'quality_group':", merged_data['quality_group'].isna().sum())


Unique values in 'water_quality':
['soft' 'salty' 'milky' 'unknown' 'fluoride' 'coloured' 'salty abandoned'
 'fluoride abandoned']

Unique values in 'quality_group':
['good' 'salty' 'milky' 'unknown' 'fluoride' 'colored']
Number of NaN values in 'water_quality': 0
Number of NaN values in 'quality_group': 0


In [24]:
# Count occurrences of 'unknown' in 'water_quality'
unknown_water_quality_count = merged_data['water_quality'].value_counts().get('unknown', 0)

# Count occurrences of 'unknown' in 'quality_group'
unknown_quality_group_count = merged_data['quality_group'].value_counts().get('unknown', 0)

print("Number of 'unknown' values in 'water_quality':", unknown_water_quality_count)
print("Number of 'unknown' values in 'quality_group':", unknown_quality_group_count)


Number of 'unknown' values in 'water_quality': 1876
Number of 'unknown' values in 'quality_group': 1876


Upon examining the unique values in the 'water_quality and 'quality_group' columns, it's apparent that both columns represent the quality of water, but 'water_quality' provides more detailed distinctions with eight unique values, while 'quality_group' aggregates these into six categories.

Considering the potential significance of the distinctions between different types of water quality, it may be beneficial to retain the column with more granular information ('wate_quality') and drop the column with fewer unique values ('quality_group'). By doing so, we maintain a more detailed representation of the types of quality of water in the dataset, which could potentially offer more insights during analysis or modeling.



In [25]:
# Drop the "quantity_group" column
merged_data.drop(columns=['quality_group'], inplace=True)

The 'recorded_by' column contains only one unique value, indicating that all records in the dataset were recorded by the same entity or group. Since this column does not offer any variability or meaningful information for analysis, it does not contribute to our predictive modeling task. Therefore, removing it simplifies the dataset without sacrificing any relevant information, making it more efficient for further analysis and model building.

In [26]:
# Drop the 'recorded_by' column
merged_data.drop(columns=['recorded_by'], inplace=True)


In [27]:
print("Unique values in 'payment':")
print(merged_data['payment'].unique())

print("\nUnique values in 'payment_type':")
print(merged_data['payment_type'].unique())
print("Number of NaN values in 'payment':", merged_data['payment'].isna().sum())
print("Number of NaN values in 'payment_type':", merged_data['payment_type'].isna().sum())


Unique values in 'payment':
['pay annually' 'never pay' 'pay per bucket' 'unknown'
 'pay when scheme fails' 'other' 'pay monthly']

Unique values in 'payment_type':
['annually' 'never pay' 'per bucket' 'unknown' 'on failure' 'other'
 'monthly']
Number of NaN values in 'payment': 0
Number of NaN values in 'payment_type': 0


In [38]:
# Drop the "payment_type" column
merged_data.drop(columns=['payment_type'], inplace=True)

Upon examination, it was clear these two columns were identical. Therefore it was appropriate to remove one column and so 'payment_type' was dropped.

In [29]:
# Get the number of unique values for each column in the merged dataset
unique_values_counts = merged_data.nunique()

# Display the number of unique values for each column
print(unique_values_counts)

id                       59400
amount_tsh                  98
date_recorded              356
gps_height                2428
longitude                57516
latitude                 57517
num_private                 65
basin                        9
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
scheme_management           12
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management_group             5
payment                      7
payment_type                 7
water_quality                8
quantity                     5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
status_group                 3
dtype: int64


In [32]:
# Unique values for source_type
print("Unique values in 'source_type':")
print(merged_data['source_type'].unique())

# Unique values for source
print("Unique values in 'source':")
print(merged_data['source'].unique())


Unique values in 'source_type':
['spring' 'rainwater harvesting' 'dam' 'borehole' 'other' 'shallow well'
 'river/lake']
Unique values in 'source':
['spring' 'rainwater harvesting' 'dam' 'machine dbh' 'other'
 'shallow well' 'river' 'hand dtw' 'lake' 'unknown']


Upon examining the unique values in the 'source_type and 'source' columns, it's apparent that both columns represent the source of water, but 'source' provides more detailed distinctions with ten unique values, while 'source_type' has only seven catagories.

Considering the potential significance of the distinctions between different sources of water, it may be beneficial to retain the column with more information ('source') and drop the column with fewer unique values ('source_type'). 


In [33]:
# Drop the 'source_type' column
merged_data.drop(columns=['source_type'], inplace=True)

In [34]:
# Get the number of unique values for each column in the merged dataset
unique_values_counts = merged_data.nunique()

# Display the number of unique values for each column
print(unique_values_counts)

id                       59400
amount_tsh                  98
date_recorded              356
gps_height                2428
longitude                57516
latitude                 57517
num_private                 65
basin                        9
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
scheme_management           12
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management_group             5
payment                      7
payment_type                 7
water_quality                8
quantity                     5
source                      10
source_class                 3
waterpoint_type              7
status_group                 3
dtype: int64


In [35]:
# Unique values in 'extraction_type'
print("Unique values in 'extraction_type':")
print(merged_data['extraction_type'].unique())

# Unique values in 'extraction_type_group'
print("\nUnique values in 'extraction_type_group':")
print(merged_data['extraction_type_group'].unique())

# Unique values in 'extraction_type_class'
print("\nUnique values in 'extraction_type_class':")
print(merged_data['extraction_type_class'].unique())


Unique values in 'extraction_type':
['gravity' 'submersible' 'swn 80' 'nira/tanira' 'india mark ii' 'other'
 'ksb' 'mono' 'windmill' 'afridev' 'other - rope pump' 'india mark iii'
 'other - swn 81' 'other - play pump' 'cemo' 'climax' 'walimi'
 'other - mkulima/shinyanga']

Unique values in 'extraction_type_group':
['gravity' 'submersible' 'swn 80' 'nira/tanira' 'india mark ii' 'other'
 'mono' 'wind-powered' 'afridev' 'rope pump' 'india mark iii'
 'other handpump' 'other motorpump']

Unique values in 'extraction_type_class':
['gravity' 'submersible' 'handpump' 'other' 'motorpump' 'wind-powered'
 'rope pump']


Upon examining the dataset, it's evident that the 'extraction_type' column provides a more detailed categorization with a wider range of variables compared to 'extraction_type_group' and 'extraction_type_class'. While 'extraction_type' contains 18 unique values, 'extraction_type_group' and 'extraction_type_class' have fewer categories, potentially resulting in loss of information. Hence, we have decided to retain 'extraction_type' and drop the other two columns for a more comprehensive analysis.


In [36]:
# Drop 'extraction_type_group' and 'extraction_type_class'
merged_data.drop(columns=['extraction_type_group', 'extraction_type_class'], inplace=True)


In [39]:
# Get the number of unique values for each column in the merged dataset
unique_values_counts = merged_data.nunique()

# Display the number of unique values for each column
print(unique_values_counts)

id                   59400
amount_tsh              98
date_recorded          356
gps_height            2428
longitude            57516
latitude             57517
num_private             65
basin                    9
region                  21
region_code             27
district_code           20
lga                    125
ward                  2092
population            1049
public_meeting           2
scheme_management       12
permit                   2
construction_year       55
extraction_type         18
management_group         5
payment                  7
water_quality            8
quantity                 5
source                  10
source_class             3
waterpoint_type          7
status_group             3
dtype: int64


In [40]:
print("Unique values in 'region_code':")
print(merged_data['region_code'].unique())

print("\nUnique values in 'district_code':")
print(merged_data['district_code'].unique())


Unique values in 'region_code':
[11 20 21 90 18  4 17 14 60 10  3 15 19 16 80  1  6  2 12 13  5  7 99 24
  9  8 40]

Unique values in 'district_code':
[ 5  2  4 63  1  8  3  6 43  7 23 33 53 62 60 30 13  0 80 67]
